In [1]:
import pandas as pd

In [6]:
import re
import string

In [15]:
from nltk.corpus import stopwords
import nltk

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
from sklearn.decomposition import NMF

In [19]:
df_union = pd.read_csv('df_union.csv')  

In [3]:
df_union

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_postal-code,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,61B3197983D553C6CCC12D3C265E4375,Mexico,19.0,18.0,30.0,28.0,AlejandraMexico,"May 10, 2009",May 2009,29721005.0,...,78258,TX,125 North Loop 1604 West,NaN,3.0,Homewood Suites by Hilton Hotel San Antonio North,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...
9996,9CE843F2646875FCB4EE774CAB6FCB68,"Detroit, Michigan",31.0,25.0,55.0,22.0,mjwingfans,"July 3, 2010",May 2010,69543852.0,...,46250,IN,6817 E. 82nd Street,NaN,2.5,Hampton Inn Indianapolis - NE / Castleton,NaN,37209.0,hotel,http://www.tripadvisor.com/Hotel_Review-g37209...
9997,7BFD165584BDE13CBD73A9AF1C1B2BF7,Austin,9.0,11.0,17.0,9.0,AustinBarb,"December 30, 2011",December 2011,122265474.0,...,78216,TX,850 Halm Boulevard,NaN,2.5,La Quinta Inn & Suites San Antonio Airport,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...
9998,7B5194085DEE03D4FCC5589C6353D683,near Liège,5.0,6.0,5.0,5.0,elpelso,"March 5, 2008",March 2008,14063603.0,...,77003,TX,1810 Bell Ave,NaN,2.5,Holiday Inn Express Houston-Downtown Conventio...,NaN,56003.0,hotel,http://www.tripadvisor.com/Hotel_Review-g56003...


## Text preprocessing

In [21]:
df_union.columns

Index(['author_id', 'author_location', 'author_num_cities',
       'author_num_helpful_votes', 'author_num_reviews',
       'author_num_type_reviews', 'author_username', 'date', 'date_stayed',
       'id', 'num_helpful_votes', 'offering_id', 'ratings_check_in_front_desk',
       'ratings_cleanliness', 'ratings_location', 'ratings_overall',
       'ratings_rooms', 'ratings_service', 'ratings_value', 'text', 'title',
       'via_mobile', 'ratings_sleep_quality',
       'ratings_business_service_(e_g_internet_access)', 'address_locality',
       'address_postal-code', 'address_region', 'address_street-address',
       'details', 'hotel_class', 'name', 'phone', 'region_id', 'type', 'url'],
      dtype='object')

In [23]:
# Compound Term Extraction
#repl = lambda x: x.replace('new york', 'new_york').replace('fashion week', 'fashion_week').replace('new year', 'new_year').replace('self care', 'self_care')

# Text preprocessing steps - remove numbers, captial letters and punctuation
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

words = set(nltk.corpus.words.words())
english_only = lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) \
         if w.lower() in words or not w.isalpha())

df_union['text'] = df_union.text.map(alphanumeric).map(punc_lower).map(english_only) #.map(repl)
df_union['title'] = df_union.title.map(alphanumeric).map(punc_lower).map(english_only) #.map(repl)
df_union.head()

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_postal-code,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...


In [16]:

words = set(nltk.corpus.words.words())

sent = "Io andiamo to the beach with my amico."
" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())
# 'Io to the beach with my'

'Io to the beach with my .'

In [9]:
df_union.text.head()

0    i recently stayed here for two nights while in...
1    i stayed here for a business trip and stayed e...
2    after a disappointing night at the hampton inn...
3    we stayed at the marriott courtyard in downtow...
4    the hotels exterior blends in well with the aw...
Name: text, dtype: object

In [24]:
# removing stop words from the text
stop = stopwords.words('english')
stop += ['.', ',', '(', ')', "'", '"']
#stop += ['http', 'https', 'co', 'see', 'editor', 'writer', 'vogue', 'best', 'look', 'one', 'may', 'make']
stop = set(stop)

## Document-Term Matrix

In [25]:
# The first document-term matrix has default Count Vectorizer values - counts of unigrams

vectorizer = CountVectorizer(stop_words = stop)

doc_word = vectorizer.fit_transform(df_union.text)

pd.DataFrame(doc_word.toarray(), columns=vectorizer.get_feature_names()).head()

,aa,aback,abandoned,abatement,abbot,abdominal,aberration,abhorrent,abide,ability,...,zinc,zip,zircon,zo,zocco,zombie,zone,zoning,zoo,zorro
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Topic modeling

### Non-Negative Matrix Factorization (NMF)

In [27]:
nmf_model = NMF(12)
doc_topic_nmf = nmf_model.fit_transform(doc_word)
doc_topic_nmf.shape

/Users/albinasitdikova/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


(10000, 12)

In [28]:
topic_word_nmf = nmf_model.components_
topic_word_nmf.shape

(12, 11266)

In [29]:
words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-11:-1]
topic_words_nmf = [[words[e] for e in l] for l in t]
topic_words_nmf

[['hotel',
  'city',
  'new',
  'times',
  'lobby',
  'square',
  'street',
  'parking',
  'right',
  'recommend'],
 ['de', 'la', 'en', 'el', 'un', 'es', 'pour', 'las', 'con', 'nous'],
 ['room',
  'bed',
  'floor',
  'bathroom',
  'view',
  'small',
  'could',
  'king',
  'size',
  'service'],
 ['das', 'die', 'ich', 'ist', 'war', 'es', 'man', 'wir', 'den', 'sind'],
 ['great',
  'location',
  'service',
  'place',
  'time',
  'really',
  'view',
  'stayed',
  'city',
  'walk'],
 ['us',
  'back',
  'told',
  'went',
  'day',
  'check',
  'said',
  'desk',
  'could',
  'service'],
 ['good',
  'breakfast',
  'location',
  'clean',
  'free',
  'area',
  'also',
  'walk',
  'well',
  'small'],
 ['la', 'di', 'per', 'un', 'non', 'che', 'con', 'si', 'da', 'al'],
 ['stay',
  'would',
  'night',
  'definitely',
  'place',
  'clean',
  'recommend',
  'really',
  'stayed',
  'comfortable'],
 ['staff',
  'friendly',
  'helpful',
  'clean',
  'well',
  'stayed',
  'comfortable',
  'time',
  'front',


In [31]:
from nltk.corpus import words

In [35]:
"che" in words.words()

True